In [ ]:
using SpecialFunctions
using PyPlot
using LinearAlgebra
using Random, Distributions
using KernelDensity
using LaTeXStrings
using JLD

In [ ]:
γ = 0.5772156649015328606065121; # Euler-Mascheroni constant
Ein(z) = (z == 0) ? 0 : γ + log(abs(z)) - expinti(-z);
k(x,y,σ) = - σ^2 * Ein(-x⋅y / σ^2);
MMD2(P,Q,σ) = mean([mean([k(x, x2,σ) for x2 in P]) - 2*mean([k(x,y,σ) for y in Q]) for x in P]) +
            mean([k(y, y2,σ) for y in Q, y2 in Q]);

In [ ]:
d = 5
D = Uniform(-1,1)
σ = 0.5
sample_sizes = [25]#, 50, 100, 200, 400]#, 100, 200]
T = 1000 # >> max num samples
N = 50

data = zeros(length(sample_sizes), N)

P = rand(D, T, d)
for (i,n) in enumerate(sample_sizes)
    println(i) 
    #Threads.@threads for j = 1:N
    for j = 1:N
        println(j)
        Pn = rand(D, n, d)
        data[i,j] = sqrt(n*MMD2(P, Pn, σ))
    end
end

In [ ]:
function log_kde(data)
    d = kde(log.(data))
    x = exp.(d.x)
    y = d.density
    area = (circshift(x, -1) .- x)[1:(end-1)]⋅y[1:(end-1)]
    return (x, y/area)
end

In [ ]:
ns = [25, 50, 100, 200]
sis = [1,3,2,4]
sigmas = [0.1, 0.2, 0.5, 1]

fig, axs = subplots(2,2, figsize=(5,4.5))
xlims = [[0,0.4], [0,0.7], [0,1.6], [0,65]]

for i = 1:length(sigmas)
    sigma = sigmas[sis[i]]
    for (j,n) in enumerate(ns)
        x,y = log_kde(data2[sis[i],j,:])
        axs[i].plot(x,y, label="n = $(ns[j])")
        axs[i].set_xlim(xlims[sis[i]])
    end
    axs[i].set_xlabel(L"$\sqrt{n} d_2^{(1)}(\,\hat{\mu}_n,\mu), \,\, \mu = \mathcal{N}_{" * "$(sigma)" * L"}$")
end
axs[1].legend()
axs[1].set_ylabel("density estimate")
axs[2].set_ylabel("density estimate")
subplots_adjust(wspace=0.3, hspace=0.4)